In [1]:
import os
import pandas as pd
import json

# Define the base directory where the folders are located (this will be replaced with your actual base directory)
base_dir = '../../Data/train/images/'

# Initialize an empty list to hold the data
data = []

# List all items in the base directory
for item in os.listdir(base_dir):
    # Construct the full path of the item
    item_path = os.path.join(base_dir, item)
    # Check if the item is a directory
    if os.path.isdir(item_path):
        # Split the directory name to get the labels
        labels = item.split('_')  # Assuming the format is 'No_Gender_Race_Age'
        # List all files in the directory
        for file in os.listdir(item_path):
            # Skip hidden files
            if file.startswith('.'):
                continue
            # Determine the mask label based on the file name
            if 'incorrect_mask' in file:
                mask_label = 1
            elif any(mask in file for mask in ['mask1', 'mask2', 'mask3', 'mask4', 'mask5']):
                mask_label = 0
            elif 'normal' in file:
                mask_label = 2
            else:
                continue  # If the file name doesn't match any condition, skip it
            # Construct the total label
            gender_label = 0 if labels[1] == "male" else 1
            age_label = 0 if int(labels[3]) < 30 else (2 if int(labels[3]) >= 60 else 1)
            total_label = 6 * mask_label + 3 * gender_label + age_label
            # Append the information to the data list
            data.append({
                'Image_path': os.path.join(item_path, file),
                'Mask_label': mask_label,
                'Gender_label': gender_label,
                'Age_label': age_label,
                'Total_label': total_label
            })

# Create a DataFrame
df = pd.DataFrame(data)
df.to_csv('./dataframe.csv', index=False)
df

,Image_path,Mask_label,Gender_label,Age_label,Total_label
0,../../Data/train/images/004421_male_Asian_30/n...,2,0,1,13
1,../../Data/train/images/004421_male_Asian_30/m...,0,0,1,1
2,../../Data/train/images/004421_male_Asian_30/m...,0,0,1,1
3,../../Data/train/images/004421_male_Asian_30/m...,0,0,1,1
4,../../Data/train/images/004421_male_Asian_30/m...,0,0,1,1
...,...,...,...,...,...
18895,../../Data/train/images/000526_female_Asian_59...,0,1,1,4
18896,../../Data/train/images/000526_female_Asian_59...,0,1,1,4
18897,../../Data/train/images/000526_female_Asian_59...,0,1,1,4
18898,../../Data/train/images/000526_female_Asian_59...,1,1,1,10


In [2]:
mask_label_count = df['Total_label'].value_counts()
print(mask_label_count)

4     4085
3     3660
0     2745
1     2050
16     817
10     817
15     732
9      732
12     549
6      549
5      545
2      415
13     410
7      410
17     109
11     109
14      83
8       83
Name: Total_label, dtype: int64


In [4]:
# Mask_label 분포 확인
mask_label_count = df['Mask_label'].value_counts()
print(mask_label_count)

# Gender_label 분포 확인
gender_label_count = df['Gender_label'].value_counts()
print(gender_label_count)

# Age_label 분포 확인
age_label_count = df['Age_label'].value_counts()
print(age_label_count)

0    13500
2     2700
1     2700
Name: Mask_label, dtype: int64
1    11606
0     7294
Name: Gender_label, dtype: int64
0    8967
1    8589
2    1344
Name: Age_label, dtype: int64


In [2]:
from PIL import Image
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter


class CustomDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        """
        Custom dataset that accepts a DataFrame, a transformation function, and returns images and labels.
        
        :param dataframe: pandas DataFrame containing the image paths and labels.
        :param transform: albumentations transformation pipeline.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)
    
    def set_transform(self, transform):
        """변환(transform)을 설정하는 메서드"""
        self.transform = transform

    def __getitem__(self, idx):
        # Retrieve image path from dataframe
        img_path = self.dataframe.iloc[idx]['Image_path']
        
        # Load image using PIL
        image = Image.open(img_path).convert('RGB')  # Convert image to RGB
        
        # Apply transformations if any
        if self.transform:
            image = self.transform(image=np.array(image))['image']  # Convert to numpy array and apply transform
        
        # Get labels from the dataframe
        mask_label = torch.tensor(self.dataframe.iloc[idx]['Mask_label'], dtype=torch.long)
        gender_label = torch.tensor(self.dataframe.iloc[idx]['Gender_label'], dtype=torch.long)
        age_label = torch.tensor(self.dataframe.iloc[idx]['Age_label'], dtype=torch.long)
        total_label = torch.tensor(self.dataframe.iloc[idx]['Total_label'], dtype=torch.long)
        
        # Return the image and the corresponding labels
        return image, mask_label, gender_label, age_label, total_label


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
basic_transform = A.Compose([   
            A.Resize(256, 256),
            A.Normalize(),
            ToTensorV2()
        ])
    
transform = A.Compose([
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])

In [4]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Total_label']) #/ Configparser로 만들자!

In [5]:
train_dataset = CustomDataset(train_df,basic_transform)
val_dataset = CustomDataset(val_df,basic_transform)

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=0,shuffle=True,pin_memory=True,drop_last=True) #/ Configparser로 만들자!
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=0,shuffle=False,pin_memory=True,drop_last=True)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from torch import nn
import timm

class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """
        # self.model = timm.create_model(model_name='mobilenetv3_small_050', pretrained=True)
        self.model = timm.create_model(model_name= 'swinv2_base_window8_256', pretrained=True)
        self.model.head.fc = nn.LazyLinear(num_classes)
        # self.model.classifier = nn.LazyLinear(num_classes)
        # for param in self.model.parameters():
        #     param.requires_grad = False

        # # classifier 의 파라미터는 훈련을 통해 업데이트되도록 설정
        # for param in self.model.head.fc.parameters():
        #     param.requires_grad = True

    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

In [7]:
model = CustomModel(18)
model = torch.nn.DataParallel(model)

/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# def train(model, optimizer,criterion, train_loader, val_loader, scheduler, device, epoch):
#     # -- logging
#     save_dir = "./output"
#     logger = SummaryWriter(log_dir=save_dir)
#     # with open(os.path.join(save_dir, "config.json"), "w", encoding="utf-8") as f:
#     #     json.dump(vars(args), f, ensure_ascii=False, indent=4)

#     best_val_acc = 0
#     best_val_loss = np.inf
#     for epoch in range(args.epochs):
#         # train loop
#         model.train()
#         loss_value = 0
#         matches = 0
#         for idx, train_batch in enumerate(train_loader):
#             inputs, labels = train_batch
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             optimizer.zero_grad()

#             outs = model(inputs)
#             preds = torch.argmax(outs, dim=-1)
#             loss = criterion(outs, labels)

#             loss.backward()
#             optimizer.step()

#             loss_value += loss.item()
#             matches += (preds == labels).sum().item()
#             if (idx + 1) % args.log_interval == 0:
#                 train_loss = loss_value / args.log_interval
#                 train_acc = matches / args.batch_size / args.log_interval
#                 current_lr = get_lr(optimizer)
#                 print(
#                     f"Epoch[{epoch}/{args.epochs}]({idx + 1}/{len(train_loader)}) || "
#                     f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
#                 )
#                 logger.add_scalar(
#                     "Train/loss", train_loss, epoch * len(train_loader) + idx
#                 )
#                 logger.add_scalar(
#                     "Train/accuracy", train_acc, epoch * len(train_loader) + idx
#                 )

#                 loss_value = 0
#                 matches = 0

#         scheduler.step()

#         # val loop
#         with torch.no_grad():
#             print("Calculating validation results...")
#             model.eval()
#             val_loss_items = []
#             val_acc_items = []
#             figure = None
#             for val_batch in val_loader:
#                 inputs, labels = val_batch
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 outs = model(inputs)
#                 preds = torch.argmax(outs, dim=-1)

#                 loss_item = criterion(outs, labels).item()
#                 acc_item = (labels == preds).sum().item()
#                 val_loss_items.append(loss_item)
#                 val_acc_items.append(acc_item)

#                 if figure is None:
#                     inputs_np = (
#                         torch.clone(inputs).detach().cpu().permute(0, 2, 3, 1).numpy()
#                     )
#                     inputs_np = dataset_module.denormalize_image(
#                         # inputs_np, dataset.mean, dataset.std
#                         inputs_np, (0.548, 0.504, 0.479), (0.237, 0.247, 0.246)
                        
#                     )
#                     figure = grid_image(
#                         inputs_np,
#                         labels,
#                         preds,
#                         n=16,
#                         shuffle=args.dataset != "MaskSplitByProfileDataset",
#                     )

#             val_loss = np.sum(val_loss_items) / len(val_loader)
#             val_acc = np.sum(val_acc_items) / len(val_set)
#             best_val_loss = min(best_val_loss, val_loss)
#             if val_acc > best_val_acc:
#                 print(
#                     f"New best model for val accuracy : {val_acc:4.2%}! saving the best model.."
#                 )
#                 torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
#                 best_val_acc = val_acc
#             torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
#             print(
#                 f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
#                 f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
#             )
#             logger.add_scalar("Val/loss", val_loss, epoch)
#             logger.add_scalar("Val/accuracy", val_acc, epoch)
#             logger.add_figure("results", figure, epoch)
#             print()

In [10]:
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

class Trainer:
    def __init__(self, model, optimizer, criterion, train_loader, val_loader, scheduler, device, args):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.device = device
        self.args = args
        self.save_dir = "./MyOutput"
        self.logger = SummaryWriter(log_dir=self.save_dir)
        self.train_loader = train_loader
        self.val_loader = val_loader
        

    def train(self):
        best_val_acc = 0
        best_val_loss = np.inf
        for epoch in range(self.args.epochs):
            # Train loop
            self.model.train()
            loss_value = 0
            matches = 0
            for idx, train_batch in tqdm(enumerate(self.train_loader)):
                inputs, labels = train_batch
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                self.optimizer.zero_grad()

                outs = self.model(inputs)
                preds = torch.argmax(outs, dim=-1)
                loss = self.criterion(outs, labels)

                loss.backward()
                self.optimizer.step()

                loss_value += loss.item()
                matches += (preds == labels).sum().item()
                if (idx + 1) % self.args.log_interval == 0:
                    train_loss = loss_value / self.args.log_interval
                    train_acc = matches / self.args.batch_size / self.args.log_interval
                    current_lr = self.get_lr(self.optimizer)
                    print(
                        f"Epoch[{epoch}/{self.args.epochs}]({idx + 1}/{len(self.train_loader)}) || "
                        f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
                    )
                    self.logger.add_scalar(
                        "Train/loss", train_loss, epoch * len(self.train_loader) + idx
                    )
                    self.logger.add_scalar(
                        "Train/accuracy", train_acc, epoch * len(self.train_loader) + idx
                    )

                    loss_value = 0
                    matches = 0

            self.scheduler.step()

            # Validation loop
            val_loss, val_acc = self.validate()

            best_val_loss = min(best_val_loss, val_loss)
            if val_acc > best_val_acc:
                print(
                    f"New best model for val accuracy : {val_acc:4.2%}! saving the best model.."
                )
                torch.save(self.model.module.state_dict(), f"{self.save_dir}/best.pth")
                best_val_acc = val_acc
            torch.save(self.model.module.state_dict(), f"{self.save_dir}/last.pth")
            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )
            self.logger.add_scalar("Val/loss", val_loss, epoch)
            self.logger.add_scalar("Val/accuracy", val_acc, epoch)
            # self.logger.add_figure("results", figure, epoch)  # Uncomment if figure is needed
            print()

    def validate(self):
        self.model.eval()
        val_loss_items = []
        val_acc_items = []

        with torch.no_grad():
            for val_batch in self.val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                outs = self.model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = self.criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(self.val_loader)
        val_acc = np.sum(val_acc_items) / len(self.args.val_set)
        return val_loss, val_acc

    def get_lr(optimizer):
        for param_group in optimizer.param_groups:
            return param_group["lr"]



In [11]:
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

class Trainer:
    def __init__(self, model, optimizer, criterion, train_loader, val_loader, scheduler, device):
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.device = device
        self.model = model.to(device)
        self.save_dir = "./MyOutput"
        self.logger = SummaryWriter(log_dir=self.save_dir)
        self.train_loader = train_loader
        self.val_loader = val_loader
        

    def train(self):
        best_val_acc = 0
        best_val_loss = np.inf
        for epoch in range(10):
            # Train loop
            self.model.train()
            loss_value = 0
            matches = 0
            for idx, train_batch in enumerate(tqdm(self.train_loader)):
                inputs, _, _, _,labels = train_batch
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                self.optimizer.zero_grad()

                outs = self.model(inputs)
                preds = torch.argmax(outs, dim=-1)
                loss = self.criterion(outs, labels)

                loss.backward()
                self.optimizer.step()

                loss_value += loss.item()
                matches += (preds == labels).sum().item()
                if (idx + 1) % 20 == 0:
                    train_loss = loss_value / 20
                    train_acc = matches / 64 / 20
                    # current_lr = self.get_lr(self.optimizer)
                    self.logger.add_scalar(
                        "Train/loss", train_loss, epoch * len(self.train_loader) + idx
                    )
                    self.logger.add_scalar(
                        "Train/accuracy", train_acc, epoch * len(self.train_loader) + idx
                    )

                    loss_value = 0
                    matches = 0


            # Validation loop
            val_loss, val_acc = self.validate()

            best_val_loss = min(best_val_loss, val_loss)
            if val_acc > best_val_acc:
                print(
                    f"New best model for val accuracy : {val_acc:4.2%}! saving the best model.."
                )
                torch.save(self.model.state_dict(), f"{self.save_dir}/best.pth")
                best_val_acc = val_acc
            torch.save(self.model.module.state_dict(), f"{self.save_dir}/last.pth")
            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )
            self.logger.add_scalar("Val/loss", val_loss, epoch)
            self.logger.add_scalar("Val/accuracy", val_acc, epoch)
            # self.logger.add_figure("results", figure, epoch)  # Uncomment if figure is needed
            
            self.scheduler.step(val_acc)

    def validate(self):
        self.model.eval()
        val_loss_items = []
        val_acc_items = []

        with torch.no_grad():
            for val_batch in tqdm(self.val_loader):
                inputs, _, _, _, labels = val_batch
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                outs = self.model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = self.criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(self.val_loader)
        val_acc = np.sum(val_acc_items) / len(self.val_loader.dataset)
        return val_loss, val_acc

    def get_lr(optimizer):
        for param_group in optimizer.param_groups:
            return param_group["lr"]

In [12]:
trainer = Trainer(model, optimizer,criterion, train_loader, val_loader, scheduler, device)
trainer.train()

  0%|          | 0/236 [00:00<?, ?it/s]

100%|██████████| 59/59 [00:38<00:00,  1.54it/s]


New best model for val accuracy : 96.88%! saving the best model..
[Val] acc : 96.88%, loss: 0.09 || best acc : 96.88%, best loss: 0.09


  3%|▎         | 7/236 [00:09<05:18,  1.39s/it]


KeyboardInterrupt: 

In [7]:
import pickle

def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# 예시: 'my_list.pkl' 파일에서 데이터 로딩
data = load_pickle('/data/ephemeral/home/level1-imageclassification-cv-05/NoBellReal/MyBase/model/yumin3/my_list.pkl')

In [8]:
data

[array([-2.3894186 ,  0.24029562, -0.41417024, -2.1218543 , -1.8421224 ,
        -2.4186127 , -3.093103  ,  0.54136896, -2.3829072 , -4.1403995 ,
        -2.6772616 , -3.8561497 , -1.6455059 ,  6.2769895 ,  2.9301379 ,
        -2.6775203 , -0.38734615, -0.88051844], dtype=float32),
 array([-0.9801132,  3.9266715,  5.96202  , -1.2968054,  1.2941788,
         1.177132 , -2.9250705, -0.7855541, -0.8700246, -3.2283187,
        -3.0922487, -2.4039738, -3.1083732, -2.7898755, -2.4794817,
        -3.6833637, -3.1402419, -3.2348363], dtype=float32),
 array([-3.0067842 ,  0.11204866, -1.4753165 , -2.5998394 , -1.2660321 ,
        -1.4564694 , -4.1348624 ,  0.92447656, -2.0823274 , -4.868037  ,
        -1.801127  , -3.0389614 , -3.0946002 ,  5.1164546 ,  4.7816505 ,
        -3.230163  , -0.01688481, -0.41265866], dtype=float32),
 array([-1.5403731 , -1.0051371 , -0.61488354, -1.7653415 , -2.3447926 ,
        -1.3932273 , -2.243364  , -0.14908099, -1.7206134 , -3.7617686 ,
        -2.8907168 , -2

In [29]:
import pandas as pd

df = pd.read_csv('/data/ephemeral/home/level1-imageclassification-cv-05/NoBellReal/MyBase/model/exp/output_soft.csv')

In [30]:
df

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,[-4.342973 5.6261725 -1.8513521 -2.563144...
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,[ 5.340122 12.687853 6.492336 2.527539...
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,[-7.4231396 2.150701 0.03014544 -3.983571...
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,[-4.6382537 2.2879758 1.2403737 0.104899...
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,[ 1.5582339 1.27466 -2.6578844 -3.551283...
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,[ 3.7446892 13.811301 5.608979 0.969739...
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,[-3.8127315 -0.35579196 -1.6724536 2.60765 ...
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,[-7.6280165 -5.5854487 -3.7951124 4.736477...
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,[ 5.9668903 13.99589 3.2857182 3.8003438 ...


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ImageID  12600 non-null  object
 1   ans      12600 non-null  object
dtypes: object(2)
memory usage: 197.0+ KB


In [32]:
import json
import numpy as np

def parse_probabilities(prob_string):
    """ 문자열 형태의 확률 데이터를 NumPy 배열로 변환합니다. """
    try:
        # JSON 형식의 문자열을 파싱
        prob_array = np.array(json.loads(prob_string))
    except json.JSONDecodeError:
        # JSON 파싱 실패 시, 수동으로 문자열을 처리
        cleaned_str = prob_string.replace('\n', '').replace('[', '').replace(']', '').replace(' ', '')
        prob_array = np.array([float(num) for num in cleaned_str.split(",")])
    return prob_array

In [16]:
df['ans'] = df['ans'].apply(parse_probabilities)
df

ValueError: could not convert string to float: '-4.3429735.6261725-1.8513521-2.5631444-6.977408-5.652109-3.45735967.679676-0.6589266-2.9874134-5.538502-3.50881310.7956604414.9114485.65852171.2463373-0.403260530.75510055'